In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch
import re

# 使用GPT2 中文模型(偏社交軟體對話)
model_name = "IDEA-CCNL/Wenzhong-GPT2-110M"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# 設定 pad_token 為 eos_token（結束符號）
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

# 清理異常符號
def clean_text(text):
    text = re.sub(r"[\u2000-\u206F\uFFF0-\uFFFF\uFE00-\uFEFF\uD800-\uDFFF]+", "", text)
    text = re.sub(r"[^\u4e00-\u9fff\u3000-\u303f\uFF00-\uFFEF\u0020-\u007E]+", "", text)
    return text.strip()

# 驗證是否為通順中文句
def is_valid_chinese(text):
    return all('\u4e00' <= char <= '\u9fff' or char in "，。！？、；：「」『』" for char in text)

# 初始輸入
text = input("請輸入文字:").strip()

# 設定參數
desired_options = 3
max_length = 40
generate_pool_size = 10  # 多產生 10 個候選句，再篩選 3 個有效的

while True:
    inputs = tokenizer(text, return_tensors="pt")

    outputs = model.generate(
        inputs["input_ids"],
        max_new_tokens=max_length,
        num_return_sequences=generate_pool_size,
        do_sample=True,
        top_k=20,
        top_p=0.92,
        temperature=0.8,
        repetition_penalty=1.1,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    last_char = text[-1] if text else ""
    seen = set()
    filtered_options = []

    for output in outputs:
        decoded = tokenizer.decode(output, skip_special_tokens=True)
        continuation = decoded[len(text):].strip()
        continuation = clean_text(continuation)

        # 避免接續重複
        if continuation.startswith(last_char):
            continuation = continuation[1:].lstrip()

        # 濾掉重複、不通順、空白
        if continuation and is_valid_chinese(continuation) and continuation not in seen:
            seen.add(continuation)
            filtered_options.append(continuation)

        if len(filtered_options) >= desired_options:
            break

    # 若不足 3 筆則補空白（可改成跳出、重新生成、或通知使用者）
    while len(filtered_options) < desired_options:
        filtered_options.append("（無法產生有效句）")

    # 顯示候選句
    print("\n請選擇以下接續句：")
    for i, option in enumerate(filtered_options):
        print(f"{i+1}: {option}")

    choice = input("\n請輸入選項編號（或輸入 exit 離開）：").strip()
    if choice.lower() == "exit":
        print("感謝使用，再見！")
        break

    if not choice.isdigit() or not (1 <= int(choice) <= desired_options):
        print("無效選項，請重新選擇。")
        continue

    selected = filtered_options[int(choice) - 1]
    if "無法產生" in selected:
        print("跳過空白選項，請重新選擇。")
        continue

    text = text.strip() + selected
    print(f"\n當前句子：{text}")


請輸入文字:我今天

請選擇以下接續句：
1: 要讲的这个故事，你一定很想知道吧但是
2: 要讲的是一种新型的打印机，它的出现主覴
3: 的职业生涯和工作经历。或许，从那时起

請輸入選項編號（或輸入 exit 離開）：1

當前句子：我今天要讲的这个故事，你一定很想知道吧但是

請選擇以下接續句：
1: 只能在过去的时间里看到现场点评。还有一
2: 可能有人会问枫叶是否也会咬住佛菩
3: 或至少在还没有看到佛陀经文之前。何况对

請輸入選項編號（或輸入 exit 離開）：1

當前句子：我今天要讲的这个故事，你一定很想知道吧但是只能在过去的时间里看到现场点评。还有一

請選擇以下接續句：
1: 丝一毫都不显露出来的。没机会了何况本就
2: 丝与之前提分手的情形不同或者说成功判断连
3: 串数字每天给体力递气的人。首先说连

請輸入選項編號（或輸入 exit 離開）：1

當前句子：我今天要讲的这个故事，你一定很想知道吧但是只能在过去的时间里看到现场点评。还有一丝一毫都不显露出来的。没机会了何况本就

請選擇以下接續句：
1: 是一对夫妻更别说被好几万人攻击。所以连
2: （無法產生有效句）
3: （無法產生有效句）

請輸入選項編號（或輸入 exit 離開）：1

當前句子：我今天要讲的这个故事，你一定很想知道吧但是只能在过去的时间里看到现场点评。还有一丝一毫都不显露出来的。没机会了何况本就是一对夫妻更别说被好几万人攻击。所以连

請選擇以下接續句：
1: 体婴儿也不愿意跟成年人一样。他们怎么能和
2: 钱部长白岩松郭晶晶也地跟大家详细描
3: 报社的喜欢也尽享不少于国内电视台的精神
